# Quantum Phase Estimation Examples

This notebook gives some examples of how to use Qubiter to write and simulate a quantum
circuit that does quantum phase estimation (qPE).

Even though qPE was invented by Kitaev, it can be understood 
(See IBM Quantum Experience Tutorial for the details) as
a quantum computer version of a much earlier model, namely, 
the von Neumann Pointer-System model for a quantum
mechanical measurement.


In the case of quantum computers, 
the Pointer in von Neumann's model is represented by several "pointer qubits"
and the System by several "system qubits".
The matrix U whose eigenvalues we wish to find
acts on the System qubits.

In Qubiter, we call the "pointer qubits" the "probes", and the qbits that U acts on the "atom qbits". We call U the "atom matrix".


First, let us load some functions that we will be calling later on.

In [3]:
from SEO_writer import *
from SEO_simulator import *
from PhaseEstSEO_writer import PhaseEstSEO_writer
from PhaseEstSEO_writer import AtomWriter
import numpy as np

## Example a: 
In this example, 2 qbits are used

probe qubits = {0}, 

atom qubits = {1}

The atom matrix is the X Pauli matrix, $U = \sigma_X$

Eigenvectors of U are 

$H|0\rangle = \frac{1}{\sqrt{2}}(|0\rangle + |1\rangle)$ for +1

$H|0\rangle = \frac{1}{\sqrt{2}}(|0\rangle - |1\rangle)$ for -1

where $H = \frac{1}{\sqrt{2}}\left[ \begin{array}{cc} 1 & 1\\ 1 & -1 \end{array} \right]$
is the Hadamard matrix



Open a writer, tell it where to write to.
We will use zero bit last (ZL) convention.

In [4]:
num_bits = 2
emb = CktEmbedder(num_bits, num_bits)
file_prefix = 'io_folder/ph_est_nb_a'
wr = SEO_writer(file_prefix, emb)

Next write the whole circuit

In [5]:
wr.write_one_bit_gate(1, OneBitGates.had2)
wr.write_one_bit_gate(0, OneBitGates.had2)

control_pos = 0
target_pos = 1
trols = Controls.new_knob(num_bits, control_pos, kind=True)
wr.write_controlled_one_bit_gate(
    target_pos, trols, OneBitGates.sigx)

wr.write_one_bit_gate(0, OneBitGates.had2)

Close English and Picture files

In [6]:
wr.close_files()

Look in files

* [./io_folder/ph_est_nb_a_2_eng.txt](./io_folder/ph_est_nb_a_2_eng.txt)
* [./io_folder/ph_est_nb_a_2_ZLpic.txt](./io_folder/ph_est_nb_a_2_ZLpic.txt)

to see the quantum circuit that was generated

Print the Picture file

In [7]:
pic_file = './io_folder/ph_est_nb_a_2_ZLpic.txt'
with open(pic_file) as f:
    print(f.read())

H   |   
|   H   
X---@   
|   H   



Specify initial state vector for simulation. 

In [8]:
init_st_vec = SEO_simulator.get_standard_basis_st([0, 0])

Open a simulator. This automatically
multiplies quantum circuit in given file.

In [9]:
sim = SEO_simulator(file_prefix, num_bits, init_st_vec)

Number of lines in file = 4

Number of Elem. Ops = 4



Print description of final state vector

In [10]:
sim.describe_fin_st(print_st_vec=True, do_pp=True, omit_zero_amps=True)

final state vector
(zero bit first in state tuple)
(0, 0) (0.707106781187+0j)
(0, 1) (0.707106781187+0j)
total probability of final state vector (=one if no measurements)= 1.0
dictionary with key=qubit, value=final (P(0), P(1))
{0: (1.0, 0.0), 1: (0.5, 0.5)}


## Example b:
In this example, num_bits are used

probe qubits = {0, 1, 2, ..., num_bits-2}

atom qubits = {num_bits-1}

The atom matrix is a Z axis rotation, $U = e^{i*rads*\sigma_Z}$, for some 
Real number $rads$

Eigenvectors of U are same as those for $\sigma_Z$, $|0\rangle$ and $|1\rangle$

An object of the AtomWriter2 class will be called by the writer of the full qPE circuit 
to write the powers of the atom matrix.

In [11]:
class AtomWriter2(AtomWriter):
    
    def __init__(self, do_write, rads, **kwargs):
        self.rads = rads
        AtomWriter.__init__(self, do_write, **kwargs)
        
    def write_pow(self, power):
        z_axis = 3
        self.write_one_bit_gate(0, OneBitGates.rot_ax, [power*self.rads, z_axis])
        
    def write_pow_hermitian(self, power):
        z_axis = 3
        self.write_one_bit_gate(0, OneBitGates.rot_ax, [-power*self.rads, z_axis])

In [12]:
for num_bits in range(2, 9):
    print('-----------------Number of bits=', num_bits)
    rads = 2*np.pi/16
    atom_wr = AtomWriter2(rads=rads, do_write=False)
    file_prefix = 'io_folder/ph_est_nb_b'
    emb = CktEmbedder(num_bits, num_bits)
    wr = PhaseEstSEO_writer(do_write=True,
                            num_probe_bits=num_bits - 1,
                            atom_writer=atom_wr,
                            file_prefix=file_prefix, 
                            emb=emb)

    wr.close_files()

    pic_file = './io_folder/ph_est_nb_b_' + str(num_bits) + '_ZLpic.txt'
    with open(pic_file) as f:
        print(f.read())

    init_st_vec = SEO_simulator.get_standard_basis_st([0]*num_bits)
    sim = SEO_simulator(file_prefix, num_bits, init_st_vec)
    sim.describe_fin_st(print_st_vec=True, do_pp=True, omit_zero_amps=True)
    
    print('spike_bit prediction=', num_bits - 1 + np.log2(rads/(2*np.pi)))

-----------------Number of bits= 2
|   H   
Rz--@   
|   H   

Number of lines in file = 3

Number of Elem. Ops = 3

final state vector
(zero bit first in state tuple)
(0, 0) (0.961939766256+0.191341716183j)
(1, 0) (0.0380602337444-0.191341716183j)
total probability of final state vector (=one if no measurements)= 1.0
dictionary with key=qubit, value=final (P(0), P(1))
{0: (0.96193976625564337, 0.038060233744356631), 1: (1.0, 0.0)}
spike_bit prediction= -3.0
-----------------Number of bits= 3
|   |   H   
|   H   |   
Rz--+---@   
Rz--@   |   
|   H   |   
|   @P--@   
|   |   H   
|   <--->   

Number of lines in file = 8

Number of Elem. Ops = 8

final state vector
(zero bit first in state tuple)
(0, 0, 0) (0.753417436516+0.503417436516j)
(0, 1, 0) (0.100135954078-0.149864045922j)
(1, 0, 0) (-0.0620757203332-0.312075720333j)
(1, 1, 0) (0.20852232974-0.0414776702601j)
total probability of final state vector (=one if no measurements)= 1.0
dictionary with key=qubit, value=final (P(0), P

We have printed the result for a range of $num\_bits$. The very
last line for each $num\_bits$ printed tells the bit where we predict the spike (P(1)>>P(0)) will occur.
If 

$rads = \frac{2 \pi}{2^r}$, 

then we predict that 

$spike\_bit + r = num\_bits -1 = num\_probes$

If $rads$ cannot be expressed in this form, then the distributions over
the first $num\_bits - 1$ bits will have multiple spikes at a bit $spike\_bit$ such that

$rads \approx 2\pi \sum_{spike\_bit} \frac{2^{spike\_bit}}{2^{num\_probes}}= 2\pi \sum_{k=0}^{num\_probes - 1} \frac{2^k}{2^{num\_probes}}n(k)$

for $0 \leq rads \leq 2\pi$, where $n(k)=1$ if $k$ is a spike bit
and $n(k)=0$ otherwise.